In [1]:
import pandas as pd

In [10]:
df_cashUC = pd.read_csv('Dataset/data_4_monthly/1_persiapan monthly track - cashUC2022.csv')
df_employee = pd.read_csv('Dataset/data_4_monthly/2_persiapan monthly track - employee2022.csv')
df_sales = pd.read_csv('Dataset/data_4_monthly/3_persiapan monthly track - Sales2022.csv')
df_transaction = pd.read_csv('Dataset/data_4_monthly/4_persiapan monthly track - Transaction2022.csv')

In [15]:
df_transaction.head()

,store_code,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,B001,NaN,NaN,NaN,NaN,NaN,"2,101","5,055","4,610","4,165","4,614","4,605","5,805"
1,B002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,679","3,439","3,765","3,772","4,285"
2,B003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,640"
3,B004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"6,726"
4,B005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5,000"


In [16]:
df_cashUC.fillna(0, inplace = True)
df_employee.fillna(0, inplace = True)
df_sales.fillna(0,inplace = True)
df_transaction.fillna(0,inplace = True)

In [21]:
month_mapping = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12,
}

def transform_row_Cash_UC(row):
    store_code = row['store_code']
    month_values = {month_mapping[month]: value for month, value in row.items() if month != 'store_code'}
    transformed_rows = [{'Year': '2022', 'Store_code': store_code, 'Month': month, 'Cash_UC': value} for month, value in month_values.items()]
    return transformed_rows

def transform_row_Employee(row):
    store_code = row['store_code']
    month_values = {month_mapping[month]: value for month, value in row.items() if month != 'store_code'}
    transformed_rows = [{'Year': '2022', 'Store_code': store_code, 'Month': month, 'Employee': value} for month, value in month_values.items()]
    return transformed_rows

def transform_row_Sales(row):
    store_code = row['store_code']
    month_values = {month_mapping[month]: value for month, value in row.items() if month != 'store_code'}
    transformed_rows = [{'Year': '2022', 'Store_code': store_code, 'Month': month, 'net_sales': value} for month, value in month_values.items()]
    return transformed_rows

def transform_row_transaction(row):
    store_code = row['store_code']
    month_values = {month_mapping[month]: value for month, value in row.items() if month != 'store_code'}
    transformed_rows = [{'Year': '2022', 'Store_code': store_code, 'Month': month, 'transaction': value} for month, value in month_values.items()]
    return transformed_rows

In [22]:
df_transformed_cash_uc = pd.DataFrame(sum(df_cashUC.apply(transform_row_Cash_UC, axis=1), []))
df_transform_employee = pd.DataFrame(sum(df_employee.apply(transform_row_Employee, axis=1), []))
df_transform_sales = pd.DataFrame(sum(df_sales.apply(transform_row_Sales, axis=1), []))
df_transform_transaction = pd.DataFrame(sum(df_transaction.apply(transform_row_transaction, axis =1), []))

In [28]:
new_df_final = pd.merge(df_transformed_cash_uc, df_transform_employee)
new_df_final2 = pd.merge(new_df_final, df_transform_sales)
new_df_final3 = pd.merge(new_df_final2, df_transform_transaction)
new_df_final3.head(10)

,Year,Store_code,Month,Cash_UC,Employee,net_sales,transaction
0,2022,B001,1,0,0.00,0,0
1,2022,B001,2,0,0.00,0,0
2,2022,B001,3,0,0.00,0,0
3,2022,B001,4,0,0.00,0,0
4,2022,B001,5,0,0.00,0,0
5,2022,B001,6,"-144,546,218",36.67,"476,257,109","2,101"
6,2022,B001,7,"24,674,654",41.28,"1,149,612,116","5,055"
7,2022,B001,8,"66,348,596",39.56,"1,000,001,047","4,610"
8,2022,B001,9,"78,860,152",37.63,"931,804,369","4,165"
9,2022,B001,10,"64,239,051",37.20,"1,013,650,015","4,614"


In [31]:
print('df_cashUC dimension = ', df_transformed_cash_uc.shape)
print('employee dimension = ', df_transform_employee.shape)
print('sales dimension = ', df_transform_sales.shape)
print('sales dimension = ', df_transform_transaction.shape)

df_cashUC dimension =  (7632, 4)
employee dimension =  (7632, 4)
sales dimension =  (7632, 4)
sales dimension =  (7632, 4)


In [32]:
new_df_final3['Year'] = new_df_final3['Year'].astype('int64')

In [33]:
if all(col in new_df_final3.columns for col in ['Year', 'Month']):
    new_df_final3['periode'] = pd.to_datetime(new_df_final3[['Year', 'Month']].assign(day=1))
else:
    print("Error: Columns 'Year' and 'Month' are not present in the DataFrame.")

In [34]:
new_df_final3.head()

,Year,Store_code,Month,Cash_UC,Employee,net_sales,transaction,periode
0,2022,B001,1,0,0.0,0,0,2022-01-01
1,2022,B001,2,0,0.0,0,0,2022-02-01
2,2022,B001,3,0,0.0,0,0,2022-03-01
3,2022,B001,4,0,0.0,0,0,2022-04-01
4,2022,B001,5,0,0.0,0,0,2022-05-01


In [37]:
new_df_final3.to_csv('Dataset/data_4_monthly/Hasil_Bersih/monthly_tracking_2022_ver3.csv', 
                     header = True, index = False)